In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [4]:
#Load the data
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Preprocess the data, drop the irrelevant data 
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [8]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [11]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [16]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [23]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [24]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [29]:
print(data.columns)

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')


In [31]:
data = pd.concat([data.reset_index(drop=True), geo_encoded_df.reset_index(drop=True)], axis=1)


In [32]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,NaN,NaN,NaN,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,NaN,NaN,NaN,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,NaN,NaN,NaN,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,NaN,NaN,NaN,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,NaN,NaN,NaN,0.0,0.0,1.0


In [33]:
data = data.dropna(axis=1, how='all')

In [34]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,NaN,NaN,NaN,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,NaN,NaN,NaN,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,NaN,NaN,NaN,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,NaN,NaN,NaN,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,NaN,NaN,NaN,0.0,0.0,1.0


In [35]:
data = data.drop(columns=['Geography'], errors='ignore')


In [36]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,NaN,NaN,NaN,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,NaN,NaN,NaN,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,NaN,NaN,NaN,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,NaN,NaN,NaN,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,NaN,NaN,NaN,0.0,0.0,1.0


In [37]:
# Step 1: Find and display duplicate columns
dupes = data.columns[data.columns.duplicated()]
print("Duplicate columns:", dupes.tolist())

# Step 2: Drop the FIRST occurrence of the duplicates (which are NaN)
data = data.loc[:, ~data.columns.duplicated(keep='last')]


Duplicate columns: ['Geography_France', 'Geography_Germany', 'Geography_Spain']


In [38]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,0.0,0.0,1.0


In [39]:
#Save the encoders and scalar
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [81]:
#Divide the dataset into dependent and independent features
X = data.drop('Exited', axis = 1)
y = data['Exited']

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

ValueError: could not convert string to float: "P'an"

In [44]:
X_train

array([[-0.70157205,  0.90488898,  0.00509723, ...,  0.99987471,
        -0.57710913, -0.57749495],
       [ 0.25470059,  0.90488898,  2.57743383, ..., -1.00012531,
        -0.57710913,  1.73161687],
       [-0.80551473,  0.90488898,  1.24362967, ...,  0.99987471,
        -0.57710913, -0.57749495],
       ...,
       [ 0.87835666, -1.10510794, -0.09017449, ...,  0.99987471,
        -0.57710913, -0.57749495],
       [ 0.17154644,  0.90488898,  0.38618414, ...,  0.99987471,
        -0.57710913, -0.57749495],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])

In [46]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

In [47]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
4,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [49]:
X_train.shape[1]

12

In [53]:
#Building my ANN model
model = Sequential([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1], )),  # HL1 connected with input layer 1
    Dense(32, activation = 'relu'), # HL2 
    Dense(1, activation = 'sigmoid') #Output
])

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.001)

In [75]:
# Compiling the model
model.compile(optimizer = opt, loss = "binary_crossentropy", metrics = ['accuracy'])

In [63]:
#Setup for tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [ ]:
#Setup Early Stopping 
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [76]:
#Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100, callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
500/500 [==============================] - 3s 5ms/step - loss: nan - accuracy: 0.3966 - val_loss: nan - val_accuracy: 0.4045
Epoch 2/100
500/500 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.3966 - val_loss: nan - val_accuracy: 0.4045


In [77]:
import numpy as np
print(np.isnan(X_train).sum(), np.isinf(X_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())


96228 0
8019 0
